In [40]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from scipy.stats import randint
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
global df_traffic, resultados, modelo, modelo_clasificacion

In [41]:
df_traffic = pd.read_csv('Classifier.csv', sep=';')
df_traffic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12283 entries, 0 to 12282
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   channelGrouping     12283 non-null  int64  
 1   date                12283 non-null  int64  
 2   fullVisitorId       12283 non-null  uint64 
 3   visitNumber         12283 non-null  int64  
 4   visitStartTime      12283 non-null  int64  
 5   browser             12283 non-null  int64  
 6   operatingSystem     12283 non-null  int64  
 7   isMobile            12283 non-null  bool   
 8   city                12283 non-null  int64  
 9   source              12283 non-null  int64  
 10  hits                12283 non-null  int64  
 11  pageviews           12283 non-null  int64  
 12  newVisits           12283 non-null  bool   
 13  transactionRevenue  12283 non-null  float64
 14  year                12283 non-null  int64  
 15  month               12283 non-null  int64  
 16  quar

In [42]:
# Dividir los datos en conjunto de entrenamiento y prueba
X = df_traffic.drop(columns=['transactionRevenue'])
y = df_traffic['transactionRevenue']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

random_forest = RandomForestRegressor()
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 358.64961081682526


In [43]:
#definiendo el K - número de subconjuntos
cv = KFold(n_splits = 5, shuffle=True)

In [44]:
# Selección de hiperparámetros para la regresión lineal
param_random_forest = {
    'n_estimators': [50, 100, 200],
    'criterion': ['mse', 'friedman_mse', 'mae'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'oob_score': [True, False]
}

In [45]:
# Configurar RandomizedSearchCV
randomized_search_rfm = RandomizedSearchCV(
    estimator=random_forest,
    param_distributions=param_random_forest,
    n_iter=10,  
    cv=5, 
    scoring='neg_mean_squared_error',  
    n_jobs=-1, 
    random_state=42  
)

In [46]:
# Ajustar el modelo
randomized_search_rfm.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['mse', 'friedman_mse',
                                                      'mae'],
                                        'max_depth': [None, 10, 20],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200],
                                        'oob_score': [True, False]},
                   random_state=42, scoring='neg_mean_squared_error')

In [47]:
# Obtener los mejores parámetros y la mejor puntuación
best_params_rf = randomized_search_rfm.best_params_
best_score_rf = randomized_search_rfm.best_score_

print("Mejores parámetros:", best_params_rf)
print("Mejor puntuación:", best_score_rf)

Mejores parámetros: {'oob_score': False, 'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'friedman_mse', 'bootstrap': False}
Mejor puntuación: -346.99562387177536


In [48]:
# Crear una nueva instancia de RandomForest con los mejores parámetros encontrados
best_rf_model = RandomForestRegressor(**randomized_search_rfm.best_params_)

# Entrenar el modelo con los datos de entrenamiento
best_rf_model.fit(X_train, y_train)

y_pred = best_rf_model.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Calcular R^2
r2 = r2_score(y_test, y_pred)
print("R^2 Score:", r2)

# Calcular RMSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)

Mean Squared Error: 292.0126625932127
R^2 Score: 0.31994728261531924
Root Mean Squared Error: 17.088377997727367


In [49]:
model_scores = {
    'model': 'RandomForestRegressor',
    'mse': mse,
    'r2': r2,
    'rmse': rmse
}
print(model_scores)

{'model': 'RandomForestRegressor', 'mse': 292.0126625932127, 'r2': 0.31994728261531924, 'rmse': 17.088377997727367}


In [50]:
def save_scores(model_scores: dict):
  status = []
  #Utilizamos un try-except en caso de que el archivo no exista que cree uno
  try:
    #Si el archivo existe continúa por acá
    scores = pd.read_csv('scores.csv', sep=';')
    status.append('El archivo existe')
    exist = model_scores['model'] in scores['model'].values
    #Validamos si el modelo ya existe en el archivo de scores
    if (exist):
      #Si el modelo ya existe reemplazamos sus valores
      status.append('El modelo existe')
      criteria = scores['model'] == model_scores['model']
      index = scores[criteria].index[0]
      scores.iloc[index] = model_scores
      status.append('Se reemplazaron los valores del modelo')
    else:
      #Si el modelo no existe lo agregamos
      status.append('El modelo no existe')
      df_model_scores = pd.DataFrame(model_scores, index=[0])
      scores = pd.concat([scores, df_model_scores], ignore_index=True)
      status.append('Se añadió modelo nuevo y sus valores')
  except:
    #Si no existe el archivo lo creamos y cargamos los scores del modelo
    status.append('El archivo no existe')
    scores = pd.DataFrame(model_scores, index=[0])

  status.append('Se sobrescrive el archivo scores.csv con valores nuevos')
  scores.to_csv('scores.csv', sep=';', index=False)
  return status

In [51]:
save_scores(model_scores)

['El archivo existe',
 'El modelo existe',
 'Se reemplazaron los valores del modelo',
 'Se sobrescrive el archivo scores.csv con valores nuevos']

In [52]:
# Guardar el modelo
import joblib
joblib.dump(best_rf_model, 'modelo_random_forest.joblib')

['modelo_random_forest.joblib']